<a href="https://colab.research.google.com/github/mohdimrandev/PDF-and-Image-Text-Extraction-Using-Google-Vision-API/blob/main/PDF_and_Image_Text_Extraction_Using_Google_Vision_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Required Libraries**

In [ ]:
!pip install fitz pymupdf pillow google-cloud-vision pandas

In [ ]:
!pip install --upgrade pymupdf

# **Set Up Google Cloud Credentials**

In [ ]:
from google.colab import files
uploaded = files.upload()
import os
credentials_file = list(uploaded.keys())[0]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_file

Saving client_secret_614198952670-84qta3hb0p1vno4tllgccfq99vsl12tj.apps.googleusercontent.com.json to client_secret_614198952670-84qta3hb0p1vno4tllgccfq99vsl12tj.apps.googleusercontent.com (1).json


# **Define Enhanced OCR Functions**

In [ ]:
import fitz  # PyMuPDF
import io
from PIL import Image
from google.cloud import vision
import pandas as pd

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

def extract_text_from_image(image_path):
    client = vision.ImageAnnotatorClient()
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    full_text = "\n".join([text.description for text in texts])
    return full_text

def extract_text(file_path):
    if file_path.lower().endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    elif file_path.lower().endswith(('.jpg', '.jpeg', '.png')):
        return extract_text_from_image(file_path)
    else:
        raise ValueError("Unsupported file format")


# **Define Bulk Processing Function**

In [ ]:
def bulk_extract_text(file_paths):
    data = []
    for file_path in file_paths:
        try:
            text = extract_text(file_path)
            data.append({"file_path": file_path, "text": text})
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    return data


# **Define Data Model Function**

In [ ]:
def create_data_model(data):
    df = pd.DataFrame(data)
    df.to_csv("extracted_data.csv", index=False)
    return df

# **Upload Documents to Colab**

In [ ]:
from google.colab import files
uploaded_files = files.upload()
file_paths = list(uploaded_files.keys())

Saving Cover Letter.pdf to Cover Letter.pdf
Saving Mohammed Imran Resume.pdf to Mohammed Imran Resume.pdf


# **Main Function to Process and Display Data**

In [ ]:
def main():
    data = bulk_extract_text(file_paths)
    df = create_data_model(data)
    print(df.head())

if __name__ == "__main__":
    main()

                   file_path  \
0           Cover Letter.pdf   
1  Mohammed Imran Resume.pdf   

                                                text  
0  Cover Letter \nDear Hiring Manager, \nI am wri...  
1  MOHAMMED IMRAN \n LinkedIn  |   +91 9080977424...  
